<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/EqualAreaHistogram_NonInteger_Edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
pio.templates.default='plotly_dark'

In [98]:
data = np.random.normal(loc=0.,scale=1., size=int(1e3))
data_sorted = np.sort(data)
n=data_sorted.size
nbins = int(2*n**(2/5))
nperbin = int(n//nbins)
x = np.arange(0,n)
edges = np.linspace(0,n,nbins+1)

In [132]:
offsets = np.arange(np.ceil(-nperbin/4),np.floor(nperbin/4))


In [133]:
indices=(edges[1:-1,None] + offsets[None,:]).astype(np.int64)

In [134]:
Y=np.take(data_sorted,indices)

In [135]:
X=np.stack((indices,np.ones_like(indices)))
X=np.moveaxis(X,(0,1,2),(2,0,1))
XT = np.moveaxis(X,(0,1,2), (0,2,1))

In [136]:
pinv=np.linalg.inv(XT@X) @ XT 
mb=np.einsum('ijk,ik->ij',pinv,Y)
smoothed_edges = np.r_[(np.min(data), mb[:,0]*edges[1:-1] + mb[:,1], np.max(data))]
height = 1/nbins/(smoothed_edges[1:]-smoothed_edges[:-1])
xy=np.zeros((height.size*2+2,2))
xy[0]=np.array([smoothed_edges[0],0])
xy[-1]=np.array([smoothed_edges[-1],0])
xy[1:-2:2]=np.c_[smoothed_edges[:-1],height]
xy[2::2]=np.c_[smoothed_edges[1:],height]

In [137]:

fig3=make_subplots()
fig3.add_scatter(x=xy[:,0], y=xy[:,1], mode='lines')
fig3.update_layout(width=800,height=600)

In [138]:
ylines=np.einsum('ijk,ik->ij', X, mb)
fig4=make_subplots()
fig4.add_scatter(x=x, y=data_sorted, mode='markers', marker_size=2)
for i,yline in enumerate(ylines):
    fig4.add_scatter(x=indices[i,:],y=yline,mode='lines')
fig4.update_layout(width=800,height=600,showlegend=False)

In [88]:
X.shape

(30, 7, 2)

In [89]:
mb.shape

(30, 2)